In [1]:
import numpy as np
import pandas as pd
import os
import sys

import random 
from timeit import default_timer as timer
import cv2
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler, Sampler
import torch.utils.data as data
import torchvision.models as models
import torch.nn as nn
from torch.nn import functional as F
import torch

from fork_of_heng_s_utility_functions import *
from heng_s_models_all import *

PI = np.pi
IMAGE_RGB_MEAN = [0.485, 0.456, 0.406]
IMAGE_RGB_STD  = [0.229, 0.224, 0.225]
DEFECT_COLOR = [(0,0,0),(0,0,255),(0,255,0),(255,0,0),(0,255,255)]
SEED = 69

In [2]:
def seed_everything(seed=SEED):
   random.seed(seed)
   os.environ['PYTHONHASHSEED'] = str(seed)
   np.random.seed(seed)
   torch.manual_seed(seed)
   torch.cuda.manual_seed(seed)
   torch.backends.cudnn.deterministic = True
seed_everything()

In [3]:
SPLIT_DIR = '../input/hengs-split'
DATA_DIR = '../input/severstal-steel-defect-detection'

In [4]:
class Logger(object):
    def __init__(self):
        self.terminal = sys.stdout  #stdout
        self.file = None

    def open(self, file, mode=None):
        if mode is None: mode ='w'
        self.file = open(file, mode)

    def write(self, message, is_terminal=1, is_file=1 ):
        if '\r' in message: is_file=0

        if is_terminal == 1:
            self.terminal.write(message)
            self.terminal.flush()
            #time.sleep(1)

        if is_file == 1:
            self.file.write(message)
            self.file.flush()

    def flush(self):
        # this flush method is needed for python 3 compatibility.
        # this handles the flush command by doing nothing.
        # you might want to specify some extra behavior here.
        pass

In [5]:
!pip install git+https://github.com/qubvel/segmentation_models.pytorch

import segmentation_models_pytorch as smp

  Cloning https://github.com/qubvel/segmentation_models.pytorch to /tmp/pip-req-build-gbchg37h
  Running command git clone -q https://github.com/qubvel/segmentation_models.pytorch /tmp/pip-req-build-gbchg37h
     |████████████████████████████████| 61kB 2.9MB/s 
  Created wheel for segmentation-models-pytorch: filename=segmentation_models_pytorch-0.0.3-cp36-none-any.whl size=29157 sha256=cc3e898587989fdb0fd040400e250e41bbd3531267f3e46b5ed5a0e35b45aa42
  Stored in directory: /tmp/pip-ephem-wheel-cache-rx76d2cy/wheels/79/3f/09/1587a252e0314d26ad242d6d2e165622ab95c95e5cfe4b942c
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp36-none-any.whl size=60963 sha256=ffac0078efb5a006425acc01fbd944ad5976461c9d2388dd3dbc7cfc5c52e030
  Stored in directory: /tmp/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.4.0-cp36-none-any.whl size=11149 sha256=88633b197c9de5210abd9fd9dc66

In [6]:
class FourBalanceClassSampler(Sampler):

    def __init__(self, dataset):
        self.dataset = dataset

        label = (self.dataset.df['Label'].values)
        label = label.reshape(-1,4)
        label = np.hstack([label.sum(1,keepdims=True)==0,label]).T

        self.neg_index  = np.where(label[0])[0]
        self.pos1_index = np.where(label[1])[0]
        self.pos2_index = np.where(label[2])[0]
        self.pos3_index = np.where(label[3])[0]
        self.pos4_index = np.where(label[4])[0]

        #assume we know neg is majority class
        num_neg = len(self.neg_index)
        self.length = 4*num_neg


    def __iter__(self):
        neg = self.neg_index.copy()
        random.shuffle(neg)
        num_neg = len(self.neg_index)

        pos1 = np.random.choice(self.pos1_index, num_neg, replace=True)
        pos2 = np.random.choice(self.pos2_index, num_neg, replace=True)
        pos3 = np.random.choice(self.pos3_index, num_neg, replace=True)
        pos4 = np.random.choice(self.pos4_index, num_neg, replace=True)

        l = np.stack([neg,pos1,pos2,pos3,pos4]).T
        l = l.reshape(-1)
        return iter(l)

    def __len__(self):
        return self.length

In [7]:
# # UNet
# def upsize(x,scale_factor=2):
#     #x = F.interpolate(x, size=e.shape[2:], mode='nearest')
#     x = F.interpolate(x, scale_factor=scale_factor, mode='nearest')
#     return x

# class Swish(nn.Module):
#     def forward(self, x):
#         return x * torch.sigmoid(x)

# class Decode(nn.Module):
#     def __init__(self, in_channel, out_channel):
#         super(Decode, self).__init__()

#         self.top = nn.Sequential(
#             nn.Conv2d(in_channel, out_channel//2, kernel_size=3, stride=1, padding=1, bias=False),
#             nn.BatchNorm2d( out_channel//2),
#             nn.ReLU(inplace=True),
#             #nn.Dropout(0.1),

#             nn.Conv2d(out_channel//2, out_channel//2, kernel_size=3, stride=1, padding=1, bias=False),
#             nn.BatchNorm2d(out_channel//2),
#             nn.ReLU(inplace=True),
#             #nn.Dropout(0.1),

#             nn.Conv2d(out_channel//2, out_channel, kernel_size=1, stride=1, padding=0, bias=False),
#             nn.BatchNorm2d(out_channel),
#             nn.ReLU(inplace=True), #Swish(), #
#         )

#     def forward(self, x):
#         x = self.top(torch.cat(x, 1))
#         return x

# class Net(nn.Module):

#     def load_pretrain(self, skip, is_print=True):
#         conversion=copy.copy(CONVERSION)
#         for i in range(0,len(conversion)-8,4):
#             conversion[i] = 'block.' + conversion[i][5:]
#         load_pretrain(self, skip, pretrain_file=PRETRAIN_FILE, conversion=conversion, is_print=is_print)

#     def __init__(self, num_class=5, drop_connect_rate=0.2):
#         super(Net, self).__init__()

#         e = ResNet18()
#         self.block = nn.ModuleList([
#            e.block0,
#            e.block1,
#            e.block2,
#            e.block3,
#            e.block4
#         ])
#         e = None  #dropped

#         self.decode1 =  Decode(512,     128)
#         self.decode2 =  Decode(128+256, 128)
#         self.decode3 =  Decode(128+128, 128)
#         self.decode4 =  Decode(128+ 64, 128)
#         self.decode5 =  Decode(128+ 64, 128)
#         self.logit = nn.Conv2d(128,num_class, kernel_size=1)

#     def forward(self, x):
#         batch_size,C,H,W = x.shape

#         #----------------------------------
#         backbone = []
#         for i in range( len(self.block)):
#             x = self.block[i](x)
#             #print(i, x.shape)

#             if i in [0,1,2,3,4]:
#                 backbone.append(x)

#         #----------------------------------
#         x = self.decode1([backbone[-1], ])                   #; print('d1',d1.size())
#         x = self.decode2([backbone[-2], upsize(x)])          #; print('d2',d2.size())
#         x = self.decode3([backbone[-3], upsize(x)])          #; print('d3',d3.size())
#         x = self.decode4([backbone[-4], upsize(x)])          #; print('d4',d4.size())
#         x = self.decode5([backbone[-5], upsize(x)])          #; print('d5',d5.size())

#         logit = self.logit(x)
#         logit = F.interpolate(logit, size=(H,W), mode='bilinear', align_corners=False)
#         return logit

In [8]:
# Class which is used by the infor object in __get_item__
class Struct(object):
    def __init__(self, is_copy=False, **kwargs):
        self.add(is_copy, **kwargs)

    def add(self, is_copy=False, **kwargs):
        #self.__dict__.update(kwargs)

        if is_copy == False:
            for key, value in kwargs.items():
                setattr(self, key, value)
        else:
            for key, value in kwargs.items():
                try:
                    setattr(self, key, copy.deepcopy(value))
                    #setattr(self, key, value.copy())
                except Exception:
                    setattr(self, key, value)

    def __str__(self):
        text =''
        for k,v in self.__dict__.items():
            text += '\t%s : %s\n'%(k, str(v))
        return text

# Creating masks
def run_length_decode(rle, height=256, width=1600, fill_value=1):
    mask = np.zeros((height,width), np.float32)
    if rle != '':
        mask=mask.reshape(-1)
        r = [int(r) for r in rle.split(' ')]
        r = np.array(r).reshape(-1, 2)
        for start,length in r:
            start = start-1  #???? 0 or 1 index ???
            mask[start:(start + length)] = fill_value
        mask=mask.reshape(width, height).T
    return mask

# Collations
def null_collate(batch):
#     pdb.set_trace()
    batch_size = len(batch)
    input = []
    truth_mask  = []
    truth_label = []
    infor = []
    for b in range(batch_size):
        input.append(batch[b][0])
        #truth_mask.append(batch[b][1])
        infor.append(batch[b][2])

        mask  = batch[b][1]
        label = (mask.reshape(4,-1).sum(1)>0).astype(np.int32)

        num_class,H,W = mask.shape
        mask = mask.transpose(1,2,0)*[1,2,3,4]
        mask = mask.reshape(-1,4)
        mask = mask.max(-1).reshape(1,H,W)

        truth_mask.append(mask)
        truth_label.append(label)

    
    input = np.stack(input)
    input = image_to_input(input, IMAGE_RGB_MEAN,IMAGE_RGB_STD)
    input = torch.from_numpy(input).float()

    truth_mask = np.stack(truth_mask)
    truth_mask = torch.from_numpy(truth_mask).long()

    truth_label = np.array(truth_label)
    truth_label = torch.from_numpy(truth_label).float()

    return input, truth_mask, truth_label, infor

# Metric
def metric_dice(logit, truth, threshold=0.1, sum_threshold=1):

    with torch.no_grad():
        probability = torch.softmax(logit,1)
        probability = one_hot_encode_predict(probability)
        truth = one_hot_encode_truth(truth)

        batch_size,num_class, H,W = truth.shape
        probability = probability.view(batch_size,num_class,-1)
        truth = truth.view(batch_size,num_class,-1)
        p = (probability>threshold).float()
        t = (truth>0.5).float()

        t_sum = t.sum(-1)
        p_sum = p.sum(-1)

        d_neg = (p_sum < sum_threshold).float()
        d_pos = 2*(p*t).sum(-1)/((p+t).sum(-1)+1e-12)

        neg_index = (t_sum==0).float()
        pos_index = 1-neg_index

        num_neg = neg_index.sum()
        num_pos = pos_index.sum(0)
        dn = (neg_index*d_neg).sum()/(num_neg+1e-12)
        dp = (pos_index*d_pos).sum(0)/(num_pos+1e-12)

        #----

        dn = dn.item()
        dp = list(dp.data.cpu().numpy())
        num_neg = num_neg.item()
        num_pos = list(num_pos.data.cpu().numpy())

    return dn,dp, num_neg,num_pos

def metric_hit(logit, truth, threshold=0.5):
    batch_size,num_class, H,W = logit.shape

    with torch.no_grad():
        logit = logit.view(batch_size,num_class,-1)
        truth = truth.view(batch_size,-1)

        probability = torch.softmax(logit,1)
        p = torch.max(probability, 1)[1]
        t = truth
        correct = (p==t)

        index0 = t==0
        index1 = t==1
        index2 = t==2
        index3 = t==3
        index4 = t==4

        num_neg  = index0.sum().item()
        num_pos1 = index1.sum().item()
        num_pos2 = index2.sum().item()
        num_pos3 = index3.sum().item()
        num_pos4 = index4.sum().item()

        neg  = correct[index0].sum().item()/(num_neg +1e-12)
        pos1 = correct[index1].sum().item()/(num_pos1+1e-12)
        pos2 = correct[index2].sum().item()/(num_pos2+1e-12)
        pos3 = correct[index3].sum().item()/(num_pos3+1e-12)
        pos4 = correct[index4].sum().item()/(num_pos4+1e-12)

        num_pos = [num_pos1,num_pos2,num_pos3,num_pos4,]
        tn = neg
        tp = [pos1,pos2,pos3,pos4,]

    return tn,tp, num_neg,num_pos

# Loss
def criterion(logit, truth, weight=None):
    logit = logit.permute(0, 2, 3, 1).contiguous().view(-1, 5)
    truth = truth.permute(0, 2, 3, 1).contiguous().view(-1)

    if weight is not None: weight = torch.FloatTensor([1]+weight).cuda()
    loss = F.cross_entropy(logit, truth, weight=weight, reduction='none')

    loss = loss.mean()
    return loss

#One-Hot for segmentation
def one_hot_encode_truth(truth, num_class=4):
    one_hot = truth.repeat(1,num_class,1,1)
    arange  = torch.arange(1,num_class+1).view(1,num_class,1,1).to(truth.device)
    one_hot = (one_hot == arange).float()
    return one_hot

def one_hot_encode_predict(predict, num_class=4):
    value, index = torch.max(predict, 1, keepdim=True)
    value  = value.repeat(1,num_class,1,1)
    index  = index.repeat(1,num_class,1,1)
    arange = torch.arange(1,num_class+1).view(1,num_class,1,1).to(predict.device)
    one_hot = (index == arange).float()
    value = value*one_hot
    return value

# Learning Rate Adjustments
def adjust_learning_rate(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def get_learning_rate(optimizer):
    lr=[]
    for param_group in optimizer.param_groups:
       lr +=[ param_group['lr'] ]

    assert(len(lr)==1) #we support only one param_group
    lr = lr[0]
    return lr

# Learning Rate Schedule
class NullScheduler():
    def __init__(self, lr=0.01 ):
        super(NullScheduler, self).__init__()
        self.lr    = lr
        self.cycle = 0

    def __call__(self, time):
        return self.lr

    def __str__(self):
        string = 'NullScheduler\n' \
                + 'lr=%0.5f '%(self.lr)
        return string

In [9]:
schduler = NullScheduler(lr=0.001)
batch_size = 4 #8
iter_accum = 8
loss_weight = None#[5,5,2,5] #
train_sampler = FourBalanceClassSampler #RandomSampler

In [10]:
class SteelDataset(Dataset):
    def __init__(self, split, csv, mode, augment=None):
#         import pdb; pdb.set_trace()
        self.split   = split
        self.csv     = csv
        self.mode    = mode
        self.augment = augment

        self.uid = list(np.concatenate([np.load(SPLIT_DIR + '/%s'%f , allow_pickle=True) for f in split]))
        df = pd.concat([pd.read_csv(DATA_DIR + '/%s'%f) for f in csv])
        df.fillna('', inplace=True)
        df['Class'] = df['ImageId_ClassId'].str[-1].astype(np.int32)
        df['Label'] = (df['EncodedPixels']!='').astype(np.int32)
        df = df_loc_by_list(df, 'ImageId_ClassId', [ u.split('/')[-1] + '_%d'%c  for u in self.uid for c in [1,2,3,4] ])
        self.df = df

    def __str__(self):
        num1 = (self.df['Class']==1).sum()
        num2 = (self.df['Class']==2).sum()
        num3 = (self.df['Class']==3).sum()
        num4 = (self.df['Class']==4).sum()
        pos1 = ((self.df['Class']==1) & (self.df['Label']==1)).sum()
        pos2 = ((self.df['Class']==2) & (self.df['Label']==1)).sum()
        pos3 = ((self.df['Class']==3) & (self.df['Label']==1)).sum()
        pos4 = ((self.df['Class']==4) & (self.df['Label']==1)).sum()

        length = len(self)
        num = len(self)*4
        pos = (self.df['Label']==1).sum()
        neg = num-pos

        #---

        string  = ''
        string += '\tmode    = %s\n'%self.mode
        string += '\tsplit   = %s\n'%self.split
        string += '\tcsv     = %s\n'%str(self.csv)
        string += '\t\tlen   = %5d\n'%len(self)
        if self.mode == 'train':
            string += '\t\tnum   = %5d\n'%num
            string += '\t\tneg   = %5d  %0.3f\n'%(neg,neg/num)
            string += '\t\tpos   = %5d  %0.3f\n'%(pos,pos/num)
            string += '\t\tpos1  = %5d  %0.3f  %0.3f\n'%(pos1,pos1/length,pos1/pos)
            string += '\t\tpos2  = %5d  %0.3f  %0.3f\n'%(pos2,pos2/length,pos2/pos)
            string += '\t\tpos3  = %5d  %0.3f  %0.3f\n'%(pos3,pos3/length,pos3/pos)
            string += '\t\tpos4  = %5d  %0.3f  %0.3f\n'%(pos4,pos4/length,pos4/pos)
        return string


    def __len__(self):
        return len(self.uid)


    def __getitem__(self, index):
        # print(index)
        folder, image_id = self.uid[index].split('/')

        rle = [
            self.df.loc[self.df['ImageId_ClassId']==image_id + '_1','EncodedPixels'].values[0],
            self.df.loc[self.df['ImageId_ClassId']==image_id + '_2','EncodedPixels'].values[0],
            self.df.loc[self.df['ImageId_ClassId']==image_id + '_3','EncodedPixels'].values[0],
            self.df.loc[self.df['ImageId_ClassId']==image_id + '_4','EncodedPixels'].values[0],
        ]
        image = cv2.imread(DATA_DIR + '/%s/%s'%(folder,image_id), cv2.IMREAD_COLOR)
        mask  = np.array([run_length_decode(r, height=256, width=1600, fill_value=1) for r in rle])

        infor = Struct(
            index    = index,
            folder   = folder,
            image_id = image_id,
        )

        if self.augment is None:
            return image, mask, infor
        else:
            return self.augment(image, mask, infor)

In [11]:
def do_valid(net, valid_loader, displays=None):
    valid_num  = np.zeros(11, np.float32)
    valid_loss = np.zeros(11, np.float32)
    
    for t, (input, truth_mask, truth_label, infor) in enumerate(valid_loader):

        #if b==5: break
        net.eval()
        input = input.cuda()
        truth_mask  = truth_mask.cuda()
        truth_label = truth_label.cuda()

        with torch.no_grad():
            logit = net(input) #data_parallel(net, input)
            loss  = criterion(logit, truth_mask)
            tn,tp, num_neg,num_pos = metric_hit(logit, truth_mask)
            dn,dp, num_neg,num_pos = metric_dice(logit, truth_mask, threshold=0.5, sum_threshold=100)
            
            #zz=0
        #---
        batch_size = len(infor)
        l = np.array([ loss.item(), tn,*tp, dn,*dp ])
        n = np.array([ batch_size, num_neg,*num_pos, num_neg,*num_pos ])
        valid_loss += l*n
        valid_num  += n

        #debug-----------------------------
        if displays is not None:
            probability = torch.sigmoid(logit)
            image = input_to_image(input, IMAGE_RGB_MEAN,IMAGE_RGB_STD)

            probability = one_hot_encode_predict(probability)
            truth_mask  = one_hot_encode_truth(truth_mask)
            
            probability_mask = probability.data.cpu().numpy()
            truth_label = truth_label.data.cpu().numpy()
            truth_mask  = truth_mask.data.cpu().numpy()

            for b in range(0, batch_size, 4):
                image_id = infor[b].image_id[:-4]
                result = draw_predict_result_label(image[b], truth_mask[b], truth_label[b], probability_mask[b], stack='vertical')
                draw_shadow_text(result,'%05d    %s.jpg'%(valid_num[0]-batch_size+b, image_id),(5,24),1,[255,255,255],2)
                image_show('result',result,resize=1)
#                 cv2.imwrite(out_dir +'/valid/%s.png'%(image_id), result)
#                 cv2.waitKey(1)
                pass
        #debug-----------------------------

        #print(valid_loss)
        print('\r %8d /%8d'%(valid_num[0], len(valid_loader.dataset)),end='',flush=True)

        pass  #-- end of one data loader --
    assert(valid_num[0] == len(valid_loader.dataset))
    valid_loss = valid_loss/valid_num

    return valid_loss

In [12]:
def run_train():
    batch_size = 4
    
    log = Logger()
    log.open('../working/log_train.txt',mode='a')
    log.write('\n--- [START ] %s\n' % ('-' * 64))
    log.write('\n')
    log.write('\tSEED         = %u\n' % SEED)
    log.write('\n')
    
    
    train_dataset = SteelDataset(
        mode    = 'train',
        csv     = ['train.csv',],
        split   = ['train_b1_11568.npy',],
        augment = train_augment,
    )
    train_loader  = DataLoader(
        train_dataset,
        #sampler     = BalanceClassSampler(train_dataset, 3*len(train_dataset)),
        #sampler    = SequentialSampler(train_dataset),
        sampler    = train_sampler(train_dataset),
        batch_size  = batch_size,
        drop_last   = True,
        num_workers = 2,
        pin_memory  = True,
        collate_fn  = null_collate
    )

    valid_dataset = SteelDataset(
        mode    = 'train',
        csv     = ['train.csv',],
        split   = ['valid_b1_1000.npy',],
        augment = valid_augment,
    )
    valid_loader = DataLoader(
        valid_dataset,
        sampler    = SequentialSampler(valid_dataset),
        #sampler     = RandomSampler(valid_dataset),
        batch_size  = 4,
        drop_last   = False,
        num_workers = 2,
        pin_memory  = True,
        collate_fn  = null_collate
    )
    
    assert(len(train_dataset)>=batch_size)
    log.write('batch_size = %d\n'%(batch_size))
    log.write('train_dataset : \n%s\n'%(train_dataset))
    log.write('valid_dataset : \n%s\n'%(valid_dataset))
    log.write('\n')
    
    log.write('** net setting **\n')
    net = smp.FPN('resnet34', encoder_weights='imagenet', classes=5)
    net = net.cuda()
    log.write('%s\n'%(type(net)))
    log.write('\tloss_weight = %s\n' % str(loss_weight))
    log.write('\ttrain_loader.sampler = %s\n' % str(train_loader.sampler))
    log.write('\n')
    
    optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, net.parameters()), lr=schduler(0), momentum=0.9, weight_decay=0.0001)

    num_iters   = 3000*20
    iter_smooth = 50
    iter_log    = 500
    iter_valid  = 1500
    iter_save   = [0, num_iters-1]\
                   + list(range(0, num_iters, 2500))#1*1000

    start_iter = 0
    start_epoch= 0
    rate       = 0
#     if initial_checkpoint is not None:
#         initial_optimizer = initial_checkpoint.replace('_model.pth','_optimizer.pth')
#         if os.path.exists(initial_optimizer):
#             checkpoint  = torch.load(initial_optimizer)
#             start_iter  = checkpoint['iter' ]
#             start_epoch = checkpoint['epoch']
#             #optimizer.load_state_dict(checkpoint['optimizer'])
#         pass
    log.write('optimizer=\n  %s\n'%(optimizer))
    log.write('schduler=\n  %s\n'%(schduler))
    log.write('\n')
    
    
    log.write('** start training here! **\n')
    log.write('   batch_size=%d,  iter_accum=%d\n'%(batch_size,iter_accum))
    log.write('                      |-------------------------------- VALID-----------------------------|---------- TRAIN/BATCH ------------------------------\n')
    log.write('rate     iter   epoch |  loss    hit_neg,pos1,2,3,4           dice_neg,pos1,2,3,4         |  loss    hit_neg,pos1,2,3,4          | time         \n')
    log.write('------------------------------------------------------------------------------------------------------------------------------------------------\n')
              #0.00000    0.0*   0.0 |  0.690   0.50 [0.00,1.00,0.00,1.00]   0.44 [0.00,0.02,0.00,0.15]  |  0.000   0.00 [0.00,0.00,0.00,0.00]  |  0 hr 00 min

    train_loss = np.zeros(20,np.float32)
    valid_loss = np.zeros(20,np.float32)
    batch_loss = np.zeros(20,np.float32)
    iter = 0
    i    = 0
    
    start = timer()
    while  iter<num_iters:
        sum_train_loss = np.zeros(20,np.float32)
        sum = np.zeros(20,np.float32)

        optimizer.zero_grad()
        
        # For one epoch i.e. all the data in one pass.
        for t, (input, truth_mask, truth_label, infor) in enumerate(train_loader):

            batch_size = len(infor)
            iter  = i + start_iter
            epoch = (iter-start_iter)*batch_size/len(train_dataset) + start_epoch
            
            # Weather to display images or not! While in validation loss
            displays = None
            #if 0:
            if (iter % iter_valid==0):
                valid_loss = do_valid(net, valid_loader, displays) # omitted outdir variable
                #pass

            if (iter % iter_log==0):
                print('\r',end='',flush=True)
                asterisk = '*' if iter in iter_save else ' '
#                 print('                      |-------------------------------- VALID-----------------------------|---------- TRAIN/BATCH ------------------------------\n')
#                 print('rate     iter   epoch |  loss    hit_neg,pos1,2,3,4           dice_neg,pos1,2,3,4         |  loss    hit_neg,pos1,2,3,4          | time         \n')
#                 print('------------------------------------------------------------------------------------------------------------------------------------------------\n')
#                 print('%0.5f  %5.1f%s %5.1f |  %5.3f   %4.2f [%4.2f,%4.2f,%4.2f,%4.2f]   %4.2f [%4.2f,%4.2f,%4.2f,%4.2f]  |  %5.3f   %4.2f [%4.2f,%4.2f,%4.2f,%4.2f]  | %s' % (\
#                          rate, iter, asterisk, epoch,
#                          *valid_loss[:11],
#                          *train_loss[:6],
#                          time_to_str((timer() - start),'min'))
#                 )
#                 print('\n')
                log.write('%0.5f  %5.1f%s %5.1f |  %5.3f   %4.2f [%4.2f,%4.2f,%4.2f,%4.2f]   %4.2f [%4.2f,%4.2f,%4.2f,%4.2f]  |  %5.3f   %4.2f [%4.2f,%4.2f,%4.2f,%4.2f]  | %s' % (\
                         rate, iter, asterisk, epoch,
                         *valid_loss[:11],
                         *train_loss[:6],
                         time_to_str((timer() - start),'min'))
                )
                log.write('\n')
                
            #if 0:
            if iter in iter_save:
                torch.save(net.state_dict(),'../working/%08d_modelFPN.pth'%(iter))
                torch.save({
                    'optimizer': optimizer.state_dict(),
                    'iter'     : iter,
                    'epoch'    : epoch,
                }, '../working/%08d_optimizerFPN.pth'%(iter))
                pass

            # learning rate schduler -------------
            lr = schduler(iter)
            if lr<0 : break
            adjust_learning_rate(optimizer, lr)
            rate = get_learning_rate(optimizer)
            
            net.train()
            input = input.cuda()
            truth_label = truth_label.cuda()
            truth_mask  = truth_mask.cuda()

            logit =  net.forward(input) #data_parallel(net,input)  
            loss = criterion(logit, truth_mask, loss_weight)
            tn,tp, num_neg,num_pos = metric_hit(logit, truth_mask)
            
            (loss/iter_accum).backward()
            if (iter % iter_accum)==0:
                optimizer.step()
                optimizer.zero_grad()

            # print statistics  ------------
            l = np.array([ loss.item(), tn,*tp ])
            n = np.array([ batch_size, num_neg,*num_pos ])

            batch_loss[:6] = l
            sum_train_loss[:6] += l*n
            sum[:6] += n
            if iter%iter_smooth == 0:
                train_loss = sum_train_loss/(sum+1e-12)
                sum_train_loss[...] = 0
                sum[...]            = 0


            print('\r',end='',flush=True)
            asterisk = ' '
            print('%0.5f  %5.1f%s %5.1f |  %5.3f   %4.2f [%4.2f,%4.2f,%4.2f,%4.2f]   %4.2f [%4.2f,%4.2f,%4.2f,%4.2f]  |  %5.3f   %4.2f [%4.2f,%4.2f,%4.2f,%4.2f]  | %s' % (\
                     rate, iter, asterisk, epoch,
                     *valid_loss[:11],
                     *train_loss[:6],
                     time_to_str((timer() - start),'min'))
            )
            print('\n')
            i=i+1
            
            # debug-----------------------------
            if 0:
                for di in range(3):
                    if (iter+di)%1000==0:

                        probability = torch.softmax(logit,1)
                        image = input_to_image(input, IMAGE_RGB_MEAN,IMAGE_RGB_STD)
                        
                        probability = one_hot_encode_predict(probability)
                        truth_mask  = one_hot_encode_truth(truth_mask)
                        
                        probability_mask = probability.data.cpu().numpy()
                        truth_label = truth_label.data.cpu().numpy()
                        truth_mask  = truth_mask.data.cpu().numpy()


                        for b in range(batch_size):
                    
                            result = draw_predict_result_label(image[b], truth_mask[b], truth_label[b], probability_mask[b], stack='vertical')

                            image_show('result',result,resize=1)
#                             cv2.imwrite('../working/%05d.png'%(di*100+b), result)
#                             cv2.waitKey(1)
                            pass
        
        pass  #-- end of one data loader --
    pass #-- end of all iterations --
    log.write('\n')

In [13]:
print('                      |-------------------------------- VALID-----------------------------|---------- TRAIN/BATCH ------------------------------\n')
print('rate     iter   epoch |  loss    hit_neg,pos1,2,3,4           dice_neg,pos1,2,3,4         |  loss    hit_neg,pos1,2,3,4          | time         \n')
print('------------------------------------------------------------------------------------------------------------------------------------------------\n')
          #0.00000    0.0*   0.0 |  0.690   0.50 [0.00,1.00,0.00,1.00]   0.44 [0.00,0.02,0.00,0.15]  |  0.000   0.00 [0.00,0.00,0.00,0.00]  |  0 hr 00 min
run_train()

                      |-------------------------------- VALID-----------------------------|---------- TRAIN/BATCH ------------------------------

rate     iter   epoch |  loss    hit_neg,pos1,2,3,4           dice_neg,pos1,2,3,4         |  loss    hit_neg,pos1,2,3,4          | time         

------------------------------------------------------------------------------------------------------------------------------------------------


--- [START ] ----------------------------------------------------------------

	SEED         = 69

batch_size = 4
train_dataset : 
	mode    = train
	split   = ['train_b1_11568.npy']
	csv     = ['train.csv']
		len   = 11568
		num   = 46272
		neg   = 39737  0.859
		pos   =  6535  0.141
		pos1  =   830  0.072  0.127
		pos2  =   224  0.019  0.034
		pos3  =  4750  0.411  0.727
		pos4  =   731  0.063  0.112

valid_dataset : 
	mode    = train
	split   = ['valid_b1_1000.npy']
	csv     = ['train.csv']
		len   =  1000
		num   =  4000
		neg   =  3440  0.860
		pos   

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /tmp/.cache/torch/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:04<00:00, 19.8MB/s]


<class 'segmentation_models_pytorch.fpn.model.FPN'>
	loss_weight = None
	train_loader.sampler = <__main__.FourBalanceClassSampler object at 0x7f7f181d3e80>

optimizer=
  SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0001
)
schduler=
  NullScheduler
lr=0.00100 

** start training here! **
   batch_size=4,  iter_accum=8
                      |-------------------------------- VALID-----------------------------|---------- TRAIN/BATCH ------------------------------
rate     iter   epoch |  loss    hit_neg,pos1,2,3,4           dice_neg,pos1,2,3,4         |  loss    hit_neg,pos1,2,3,4          | time         
------------------------------------------------------------------------------------------------------------------------------------------------
0.00000    0.0*   0.0 |  4.646   0.02 [0.06,0.26,0.24,0.20]   0.05 [0.04,0.02,0.09,0.13]  |  0.000   0.00 [0.00,0.00,0.00,0.00]  |  0 hr 00 min
0.00100    0.0    0.0 |  4.646   0

In [14]:
os.listdir('../working')

['00030000_modelFPN.pth',
 '00005000_optimizerFPN.pth',
 '00027500_optimizerFPN.pth',
 '00007500_modelFPN.pth',
 'log_train.txt',
 '00050000_optimizerFPN.pth',
 '00052500_optimizerFPN.pth',
 '00055000_modelFPN.pth',
 '00047500_modelFPN.pth',
 '00042500_optimizerFPN.pth',
 '00025000_modelFPN.pth',
 '00027500_modelFPN.pth',
 '00002500_modelFPN.pth',
 '00007500_optimizerFPN.pth',
 '00030000_optimizerFPN.pth',
 '00055000_optimizerFPN.pth',
 '00005000_modelFPN.pth',
 '00059999_optimizerFPN.pth',
 '00020000_modelFPN.pth',
 '00015000_modelFPN.pth',
 '00025000_optimizerFPN.pth',
 '00017500_modelFPN.pth',
 '00010000_optimizerFPN.pth',
 '00037500_optimizerFPN.pth',
 '00020000_optimizerFPN.pth',
 '00040000_optimizerFPN.pth',
 '00022500_modelFPN.pth',
 '00050000_modelFPN.pth',
 '00042500_modelFPN.pth',
 '00035000_optimizerFPN.pth',
 '00017500_optimizerFPN.pth',
 '__output__.json',
 '00037500_modelFPN.pth',
 '00032500_optimizerFPN.pth',
 '00047500_optimizerFPN.pth',
 '00000000_optimizerFPN.pth',
 '